In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import tensorflow_datasets as tfds

# Load the IMDB dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True, split='train')

import pandas as pd

def dataset_to_dataframe(dataset):
    texts = []
    sentiments = []

    for text, label in dataset:
        texts.append(text.numpy().decode('utf-8'))
        sentiments.append(label.numpy())

    df = pd.DataFrame({
        'text': texts,
        'sentiment': sentiments
    })
    return df

# Convert the dataset
df = dataset_to_dataframe(dataset)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LC32JD_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LC32JD_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LC32JD_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
>>> import nltk
>>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Initialize the PorterStemmer
stemmer = PorterStemmer()

stop_words = ENGLISH_STOP_WORDS
punctuation_chars = string.punctuation + '-'
table = str.maketrans('', '', punctuation_chars)

sentences = []

for sentence in df['text'].fillna('').str.lower():
    # Check if the sentence is not NaN (float in pandas)
    if isinstance(sentence, str):
        soup = BeautifulSoup(sentence, 'html.parser')
        sentence = soup.get_text()
        words = word_tokenize(sentence)

        # Remove stop words
        words = [word for word in words if word.lower() not in stop_words]

        # Apply stemming and remove punctuation
        filtered_sentence = " ".join([stemmer.stem(word).translate(table) for word in words])
        sentences.append(filtered_sentence)
    else:
        sentences.append("")  # Replace NaN with an empty string

# Remove 'text' from sentences (if it exists)
sentences = [s.replace('text ', '') for s in sentences]

# Add the preprocessed sentences back to the DataFrame
df['text'] = sentences

<ipython-input-5-d6662a53752f>:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence, 'html.parser')


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [7]:
 df['sentiment'].value_counts()

sentiment
0    12500
1    12500
Name: count, dtype: int64

In [8]:

# Separate the body and label columns into x and y
X = df['text']
y = df['sentiment']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create a pipeline to use TF-IDF vectorizer and then train a Support Vector Classifier
pipeline = Pipeline([('tfidf', TfidfVectorizer(min_df=5)), ('clf', LogisticRegression(C=10))])
# Train the pipeline on the training data
pipeline.fit(X_train, y_train)
# Test the pipeline on the training and testing sets
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)
# Evaluate the accuracy of the pipeline on the training and testing sets
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)
# Print the accuracy of the pipeline on the training and testing sets
print('Accuracy on training set:', accuracy_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy on training set: 0.98025


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [10]:
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm="l2"), LogisticRegression())
# running the grid search takes a long time because of the
# relatively large grid and the inclusion of trigrams
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
 "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]}
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best cross-validation score: 0.89
Best parameters:
{'logisticregression__C': 10, 'tfidfvectorizer__ngram_range': (1, 2)}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
import joblib
joblib.dump(grid, '/content/drive/MyDrive/sentiment_analysis_pipeline.joblib')

['/content/drive/MyDrive/sentiment_analysis_pipeline.joblib']